In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
from helpers import *
import datetime
import pandas as pd

ImportError: No module named 'costs'

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = 'train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [3]:
tX.shape, y.shape

((250000, 30), (250000,))

In [5]:
y=y.reshape(len(y),1)
y.shape

(250000, 1)

In [6]:
tX

array([[ 138.47 ,   51.655,   97.827, ...,    1.24 ,   -2.475,  113.497],
       [ 160.937,   68.768,  103.235, ..., -999.   , -999.   ,   46.226],
       [-999.   ,  162.172,  125.953, ..., -999.   , -999.   ,   44.251],
       ..., 
       [ 105.457,   60.526,   75.839, ..., -999.   , -999.   ,   41.992],
       [  94.951,   19.362,   68.812, ..., -999.   , -999.   ,    0.   ],
       [-999.   ,   72.756,   70.831, ..., -999.   , -999.   ,    0.   ]])

In [72]:
df= pd.read_csv(DATA_TRAIN_PATH)
#df

In [71]:
#df.iloc[:,18:]

In [70]:
#df['PRI_lep_pt']

Calcul de moyennes. Il faut faire une fonction !

In [82]:
#st_tX, n_mean, n_std= standardize(tX, mean_x=None, std_x=None)

On standardise cet array

In [37]:
#std_tX, new_mean, new_std= standardize(new_tX, mean_x=None, std_x=None)

In [103]:
#print(std_tX.shape, new_mean, new_std)

# First method
We remove all columns containing -999. using remove_outlier_columns function.

In [7]:
new_tX, new_mean, new_std = standardize(remove_outlier_columns(tX), mean_x=None, std_x=None)
new_tX.shape

(250000, 20)

# Second method
We transform all the -999. values by the mean of the columns (calculated without counting the -999.)

In [19]:
mean_tX, mean_mean, mean__std = standardize(undefToMeanMean(tX), mean_x=None, std_x=None)
mean_tX.shape

(250000, 31)

# Linear regression
### with gradient descent method

In [11]:
# Define the parameters of the algorithm.
max_iters = 500
gamma = 0.04
initial_w = np.zeros((new_tX.shape[1], 1))
gd_losses, gd_ws, gd_loss, gd_w = least_square_GD(y, new_tX, initial_w, gamma,max_iters)

Gradient Descent(499/499): loss=0.3527203664053023


# Linear regression
### with stochastique gradient descent

In [9]:
# Define the parameters of the algorithm.
max_iters = 1
batch_size=1

# For the moment we use this gamma
gamma = 0.04

initial_w = np.zeros((new_tX.shape[1], 1))

# Start SGD.
start_time = datetime.datetime.now()
sgd_losses, sgd_ws, sgd_loss, sgd_w  = least_square_SGD(y, new_tX, batch_size, initial_w, max_iters, 500 , gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("SGD: execution time={t:.3f} seconds".format(t=exection_time))

0
Gradient Descent(0/0): loss=2.5131890928318876
SGD: execution time=3.220 seconds


Attention a retirer la premiere coordonnée de y avant de l'utiliser !

# Least square method

In [23]:
#LS_weight, LS_mse= least_squares(y,st_tX)
#print(LS_weight)
#print(LS_mse)

On retire w0

In [7]:
#TLS_weight=LS_weight[1:]

# Ridge Regression

In [86]:
lambda_=0.1
w, RR_mse = ridge_regression(y, new_tX, lambda_)
w.shape, RR_mse

((20,), 0.36119518034873893)

In [87]:
true_w = w[1:]

# Logistic Regression
### with gradient descent method

ONLY FOR LOGISTIC REGRESSION:
y take only 1 and 0 values

In [6]:
for i in range(len(y)):
    if y[i]==-1:
        y[i]=0

We define some parameters.

In [7]:
max_iters = 500
threshold = 1e-8
alpha = 0.000001
initial_w = np.zeros((new_tX.shape[1], 1))


In [90]:
l,w = logistic_regression(y, new_tX, initial_w, alpha, max_iters, threshold, True)

Current iteration=0, the loss=173286.79513998624
Current iteration=100, the loss=131993.09792831203
Current iteration=200, the loss=130424.29482058283
Current iteration=300, the loss=129735.52048552141
Current iteration=400, the loss=129385.29307579162
The loss=129194.80961871588


### with newton's method

In [31]:
#l,w = logistic_regression(y, new_tX, initial_w, alpha, max_iters, threshold, False)

In [8]:
w=initial_w

In [23]:
sig=(sigmoid(np.dot(new_tX,w))*(1-sigmoid(np.dot(new_tX,w))))[:,0]
sig

array([ 0.25,  0.25,  0.25, ...,  0.25,  0.25,  0.25])

In [25]:
np.diag(sig).shape

(250000, 250000)

In [ ]:
np.dot(new_tX.T,S)

In [40]:
#calculate_logistic_hessian(y, new_tx, initial_w)

# Penalized Logistic Regression
### with gradient descent

In [6]:
for i in range(len(y)):
    if y[i]==-1:
        y[i]=0

Define some parameters

In [7]:
max_iter = 1000
alpha = 0.00001
lambda_ = 0.01
threshold = 1e-8
initial_w = np.zeros((new_tX.shape[1], 1))

In [8]:
reg_l, reg_w = reg_logistic_regression(y, new_tX, initial_w, lambda_, alpha, max_iter, threshold)

Current iteration=0, the loss=173286.79513998624
Current iteration=50, the loss=140525.779874148
Current iteration=100, the loss=138115.04928956908
Current iteration=150, the loss=137541.8517885063
Current iteration=200, the loss=137382.66457474587
Current iteration=250, the loss=137335.24286073857
Current iteration=300, the loss=137320.7034387231
Current iteration=350, the loss=137316.19412725815
Current iteration=400, the loss=137314.78914766252
Current iteration=450, the loss=137314.35057673912
Current iteration=500, the loss=137314.21356883264
Current iteration=550, the loss=137314.1707539051
Current iteration=600, the loss=137314.1573722657
Current iteration=650, the loss=137314.15318958656
Current iteration=700, the loss=137314.15188216226
Current iteration=750, the loss=137314.15147347716
Current iteration=800, the loss=137314.15134572494
Current iteration=850, the loss=137314.15130579
Current iteration=900, the loss=137314.15129330597
Current iteration=950, the loss=137314.1512

## Do your thing crazy machine learning thing here :) ...

## Generate predictions and save ouput in csv format for submission:

In [88]:
DATA_TEST_PATH = 'test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)


Pour la méthode 1:

In [91]:
#new_tX_test = remove_outlier_columns(tX_test)

In [93]:
OUTPUT_PATH = 'hbbbbtrue.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(true_w, new_tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)